In [ ]:
# Create PYMC model
# Despite altering the priors the estimates for Home theta and away theta
# This model struggled with convergence
# It may be becayse the attacking parameters are set around 25 and defence 0
# Regression seems to converge better when parameters centered around 0

import pymc as pm

# Create Model
with pm.Model() as model:
    # home advantage - changed this to Normal Distribution so could get prior predictive
    #home = pm.Flat("home")
    home = pm.Normal("home", 0, 10)

    # attack ratings for team - common mu and sd that all teams draw from
    atts_mu = pm.Normal("atts_mu", 25, 10)
    atts_sd = pm.HalfNormal("atts_sd", 10)
    atts = pm.Normal("atts", mu=atts_mu, sigma=atts_sd, shape=n_teams)

    # defence ratings for team - common mu and sd that all teams draw from
    defs_mu = pm.Normal("defs_mu", 0, 10)
    defs_sd = pm.HalfNormal("defs_sd", 10)
    defs = pm.Normal("defs", mu=defs_mu, sigma=defs_sd , shape=n_teams)

    # calulate theta - keep as positive - more positive will add more points
    home_theta = home + atts[home_team] + defs[away_team]
    away_theta = atts[away_team] + defs[home_team]

    #  Priors on sd - can this be edited to have a different one for each team
    sigma_home_points = pm.HalfNormal("sigma_home_points", sigma=10)
    sigma_away_points = pm.HalfNormal("sigma_away_points", sigma=10)

    # goal expectation - this doesn't go to Posterior like the rest
    home_points = pm.Truncated("home_points",
        pm.Normal.dist(mu=home_theta, sigma=sigma_home_points), lower=0,observed=home_score_arr)
    away_points = pm.Truncated("away_points",
        pm.Normal.dist(mu=away_theta, sigma=sigma_away_points), lower=0,observed=away_score_arr)
    

# Get prior predictive model
with model:
    prior_samples = pm.sample_prior_predictive(1000)








In [ ]:
# Create PYMC model
# Despite altering the priors the estimates for Home theta and away theta

import pymc as pm

# Create Model
with pm.Model() as model:
    # home advantage - changed this to Normal Distribution so could get prior predictive
    home = pm.Normal("home", 0, 0.1)

    # attack ratings for team - common mu and sd that all teams draw from
    atts_mu = pm.Normal("atts_mu", 0, 0.01)
    atts_sd = pm.HalfNormal("atts_sd", 0.01)
    atts = pm.Normal("atts", mu=atts_mu, sigma=atts_sd, shape=n_teams)

    # defence ratings for team - common mu and sd that all teams draw from
    defs_mu = pm.Normal("defs_mu", 0, 0.01)
    defs_sd = pm.HalfNormal("defs_sd", 0.01)
    defs = pm.Normal("defs", mu=defs_mu, sigma=defs_sd , shape=n_teams)

    # calulate theta 
    # There is exponential growth in away theta - this caused issues
    home_theta = pm.math.exp(home + atts[home_team] + defs[away_team])
    away_theta = pm.math.exp(atts[away_team] + defs[home_team])

    # Register away_theta and home_theta as a deterministic variable
    away_theta_det = pm.Deterministic("away_theta", away_theta)
    home_theta_det = pm.Deterministic("home_theta", away_theta)

    #  Priors on sd - can this be edited to have a different one for each team
    sigma_home_points = pm.HalfNormal("sigma_home_points", sigma=1)
    sigma_away_points = pm.HalfNormal("sigma_away_points", sigma=1)

    # goal expectation - this doesn't go to Posterior like the rest
    home_points = pm.Truncated("home_points",
        pm.Normal.dist(mu=home_theta, sigma=sigma_home_points), lower=0,observed=home_score_arr)
    away_points = pm.Truncated("away_points",
        pm.Normal.dist(mu=away_theta, sigma=sigma_away_points), lower=0,observed=away_score_arr)
    
    
    # Get some prior samples
    prior_samples = pm.sample_prior_predictive(1000)
    


# Plot chart
import matplotlib.pyplot as plt
import arviz as az

# The below range looks more appropriate - however there is still negative values

set_obs_number = 1

# Extract the value for the line from away_score_arr[0]
value = away_score_arr[set_obs_number]

# Plot simulated values for first observation
az.plot_dist(
    prior_samples.prior_predictive['away_points'][:,:,set_obs_number].values,
    kind="hist",
    hist_kwargs={"alpha": 0.6},
    label="simulated",
)

# Add a vertical line for away_score_arr[0]
plt.axvline(x=value, color='red', linestyle='--', label=f"Actual data")

# Add labels and legend
plt.xlabel("Away Points")
plt.ylabel("Frequency")
plt.legend()
plt.title(f"Actual data: {value}")
plt.show()


# Plot simulated values for Away theta for first observation
# This value is far too wide - need a smaller sd of the mean
az.plot_dist(
    prior_samples.prior["away_theta"][:,:,set_obs_number].values,
    kind="hist",
    hist_kwargs={"alpha": 0.6},
    label="simulated",
)
plt.xlabel("Away theta")
plt.title("Away theta simulated for 1 obs")
plt.show()


# Plot all observations
# Plot simulated values for first observation
az.plot_dist(
    prior_samples.prior_predictive['away_points'][:,:,set_obs_number].values,
    kind="hist",
    hist_kwargs={"alpha": 0.6,"density": True},
    label="simulated",
)

az.plot_dist(
    away_score_arr,
    kind="hist",
    color="C1",
    hist_kwargs={"alpha": 0.6,"density": True},
    label="Actual",
)


# Add labels and legend
plt.title("All obs")
plt.xlabel("Away Points")
plt.ylabel("Frequency")
plt.legend()
plt.show()





In [ ]:
# Create PYMC model
# Despite altering the priors the estimates for Home theta and away theta

import pymc as pm

# Create Model
with pm.Model() as model:
    
    # home advantage - changed this to Normal Distribution so could get prior predictive
    home = pm.Normal("home", 0, 0.1)

    # attack ratings for team 
    atts_mu = pm.Normal("atts_mu", 0, 1)
    atts_sd = pm.HalfNormal("atts_sd", 0.1)
    atts = pm.Normal("atts", mu=atts_mu, sigma=atts_sd, shape=n_teams)

    # defence ratings for team 
    defs_mu = pm.Normal("defs_mu", 0, 1)
    defs_sd = pm.HalfNormal("defs_sd", 0.1)
    defs = pm.Normal("defs", mu=defs_mu, sigma=defs_sd , shape=n_teams)

    # calulate theta for home and away - this is mean expected points
    # LogNormal is constrained to positive values- but can also control sigma
    home_theta = pm.LogNormal("home_theta", mu= home + atts[home_team] + defs[away_team], sigma=0.5)
    away_theta = pm.LogNormal("away_theta", mu= atts[away_team] + defs[home_team], sigma=1)


    #  Priors on sd - can this be edited to have a different one for each team
    sigma_home_points = pm.HalfNormal("sigma_home_points", sigma=5)
    sigma_away_points = pm.HalfNormal("sigma_away_points", sigma=5)

    # goal expectation - this doesn't go to Posterior like the rest
    # Even with positive expectation - still need to keep truncated here
    # E.g. could have expected 10, sd 5 and get a negative
    home_points = pm.Truncated("home_points",
        pm.Normal.dist(mu=home_theta, sigma=sigma_home_points), lower=0,observed=home_score_arr)
    away_points = pm.Truncated("away_points",
        pm.Normal.dist(mu=away_theta, sigma=sigma_away_points), lower=0,observed=away_score_arr)
    
    
    # Get some prior samples
    prior_samples = pm.sample_prior_predictive(1000)
    


# Plot chart
import matplotlib.pyplot as plt
import arviz as az

# The below range looks more appropriate - however there is still negative values

set_obs_number = 1

# Extract the value for the line from away_score_arr[0]
value = away_score_arr[set_obs_number]

# Plot simulated values for first observation
az.plot_dist(
    prior_samples.prior_predictive['away_points'][:,:,set_obs_number].values,
    kind="hist",
    hist_kwargs={"alpha": 0.6},
    label="simulated",
)

# Add a vertical line for away_score_arr[0]
plt.axvline(x=value, color='red', linestyle='--', label=f"Actual data")

# Add labels and legend
plt.xlabel("Away Points")
plt.ylabel("Frequency")
plt.legend()
plt.title(f"Actual data: {value}")
plt.show()


# Plot simulated values for Away theta for first observation
# This value is far too wide - need a smaller sd of the mean
az.plot_dist(
    prior_samples.prior["away_theta"][:,:,set_obs_number].values,
    kind="hist",
    hist_kwargs={"alpha": 0.6},
    label="simulated",
)
plt.xlabel("Away theta")
plt.title("Away theta simulated for 1 obs")
plt.show()


# Plot all observations
# Plot simulated values for first observation
az.plot_dist(
    prior_samples.prior_predictive['away_points'][:,:,set_obs_number].values,
    kind="hist",
    hist_kwargs={"alpha": 0.6,"density": True},
    label="simulated",
)

az.plot_dist(
    away_score_arr,
    kind="hist",
    color="C1",
    hist_kwargs={"alpha": 0.6,"density": True},
    label="Actual",
)


# Add labels and legend
plt.title("All obs")
plt.xlabel("Away Points")
plt.ylabel("Frequency")
plt.legend()
plt.show()





In [ ]:
# Now want to generate values from model
# And compare to actual data
# Averaging all draws and chains

def point_expectation(trace_posterior, home_team_id, away_team_id):   
    # get parameters
    home = np.mean(trace_posterior["home"])
    atts_home = np.mean(trace_posterior["atts"][:,:,home_team_id]) 
    atts_away = np.mean(trace_posterior["atts"][:,:,away_team_id])
    defs_home = np.mean(trace_posterior["defs"][:,:,home_team_id]) 
    defs_away = np.mean(trace_posterior["defs"][:,:,away_team_id]) 
    print('home ', home.values)
    print('atts_home ', atts_home.values)
    print('atts away ',atts_away.values)
    print('defs home ',defs_home.values)
    print('defs away ',defs_away.values)
                

    # calculate theta - try adjusting to negative
    home_theta = home + atts_home + defs_away
    away_theta = atts_away + defs_home

    # return the average points scored per team
    return home_theta.values, away_theta.values

In [ ]:
# There is 0 in away points so Model would not converge 

import pymc as pm

# Create Model
with pm.Model() as model:
    home = pm.Normal("home", 0, 0.5)

    # attack ratings for team - common mu and sd that all teams draw from
    atts_mu = pm.Normal("atts_mu", 0, 0.5)
    atts_sd = pm.HalfNormal("atts_sd", 0.5)
    atts = pm.Normal("atts", mu=atts_mu, sigma=atts_sd, shape=n_teams)

    # defence ratings for team - common mu and sd that all teams draw from
    defs_mu = pm.Normal("defs_mu", 0, 0.5)
    defs_sd = pm.HalfNormal("defs_sd", 0.5)
    defs = pm.Normal("defs", mu=defs_mu, sigma=defs_sd , shape=n_teams)

    # calulate theta
    home_theta = home + atts[home_team] + defs[away_team]
    away_theta = atts[away_team] + defs[home_team]

    # Register away_theta and home_theta as a deterministic variable
    away_theta_det = pm.Deterministic("away_theta", away_theta)
    home_theta_det = pm.Deterministic("home_theta", away_theta)

     
    sigma_home_points = pm.HalfNormal("sigma_home_points", sigma=0.5)
    sigma_away_points = pm.HalfNormal("sigma_away_points", sigma=0.5)

    # goal expectation - this doesn't go to Posterior like the rest
    # Log Normal can't be used as 
    home_points = pm.LogNormal("home_points", mu= home_theta, sigma=sigma_home_points,observed=home_score_arr)
    away_points = pm.LogNormal("away_points", mu= away_theta, sigma=sigma_away_points,observed=away_score_arr)

    # Get prior samples
    prior_samples = pm.sample_prior_predictive(1000)
    



In [ ]:
# Plot a Log Normal Distribution

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import lognorm

# Parameters for the log-normal distribution
sigma = 0.5  # Standard deviation in log-space
log_sigma = np.log(sigma)
print(sigma)
print(log_sigma)

mu = 3     # Mean in log-space

median_orig = np.exp(mu)
# Compute mean in the original space
mean_original = np.exp(mu + sigma**2 / 2)

# Compute variance in the original space
variance_original = (np.exp(sigma**2) - 1) * np.exp(2 * mu + sigma**2)

# Compute standard deviation in the original space
std_original = np.sqrt(variance_original)

print('mu log: ',mu)
print('median orig: ', median_orig)
print('mean orig: ', mean_original)
print('std orig: ',std_original)


# Generate x values (original space)
# Start, stop, number
x = np.linspace(0, 100, 100)  # Avoid 0 since log-normal is undefined there

# Compute the probability density function (PDF)
pdf = lognorm.pdf(x, s=sigma, scale=np.exp(mu))  # scale = exp(mu)

# Plot the log-normal distribution
plt.figure(figsize=(8, 5))
plt.plot(x, pdf, label=f"LogNormal(mu={mu}, sigma={sigma})")
plt.title("Log-Normal Distribution")
plt.xlabel("x")
plt.ylabel("Probability Density")
plt.legend()
plt.grid(True)
plt.show()




In [ ]:
# Latest working model

import pymc as pm

# Create Model
with pm.Model() as model:
    home = pm.Normal("home", 0, 10)

    # attack ratings for team - common mu and sd that all teams draw from
    atts_mu = pm.Normal("atts_mu", 0, 10)
    atts_sd = pm.HalfNormal("atts_sd", 10)
    atts = pm.Normal("atts", mu=atts_mu, sigma=atts_sd, shape=n_teams)

    # defence ratings for team - common mu and sd that all teams draw from
    defs_mu = pm.Normal("defs_mu", 0, 10)
    defs_sd = pm.HalfNormal("defs_sd", 10)
    defs = pm.Normal("defs", mu=defs_mu, sigma=defs_sd , shape=n_teams)

    # calulate theta
    home_theta = home + atts[home_team] + defs[away_team]
    away_theta = atts[away_team] + defs[home_team]

    sigma_home_points = pm.HalfNormal("sigma_home_points", sigma= 10)
    sigma_away_points = pm.HalfNormal("sigma_away_points", sigma= 10)

    # Change back to truncated Normal
    home_points = pm.Truncated("home_points",
        pm.Normal.dist(mu=home_theta, sigma=sigma_home_points), lower=0,observed=home_score_arr)
    away_points = pm.Truncated("away_points",
        pm.Normal.dist(mu=away_theta, sigma=sigma_away_points), lower=0,observed=away_score_arr)
    
    # Get prior samples
    prior_samples = pm.sample_prior_predictive(1000)
    



In [ ]:
# Archive code trying to model sigma just by using Home team and away team
# Had lots of divergences

import pymc as pm

# Create Model
with pm.Model() as model:
    home = pm.Normal("home", 0, 10)

    # attack ratings for team - common mu and sd that all teams draw from
    atts_mu = pm.Normal("atts_mu", 0, 10)
    atts_sd = pm.HalfNormal("atts_sd", 10)
    atts = pm.Normal("atts", mu=atts_mu, sigma=atts_sd, shape=n_teams)

    # defence ratings for team - common mu and sd that all teams draw from
    defs_mu = pm.Normal("defs_mu", 0, 10)
    defs_sd = pm.HalfNormal("defs_sd", 10)
    defs = pm.Normal("defs", mu=defs_mu, sigma=defs_sd , shape=n_teams)

    # calulate theta
    home_theta = home + atts[home_team] + defs[away_team]
    away_theta = atts[away_team] + defs[home_team]

    # Allow sigma_home_points and sigma_away_points to vary for each team
    # Not sure if the sigma should take into account the above factors as well
    # E.g if we know 2 teams are evenly matched then their sigmas should be lower
    sigma_home_points_team = pm.HalfNormal("sigma_home_points_team", sigma=10, shape=n_teams)
    sigma_away_points_team = pm.HalfNormal("sigma_away_points_team", sigma=10, shape=n_teams)

    # Index the sigmas based on the team
    sigma_home_points = sigma_home_points_team[home_team]
    sigma_away_points = sigma_away_points_team[away_team]


    # Truncated Normal for observed scores
    home_points = pm.Truncated(
        "home_points",
        pm.Normal.dist(mu=home_theta, sigma=sigma_home_points),
        lower=0,
        observed=home_score_arr,
    )
    away_points = pm.Truncated(
        "away_points",
        pm.Normal.dist(mu=away_theta, sigma=sigma_away_points),
        lower=0,
        observed=away_score_arr,
    )
    
    # Get prior samples
    prior_samples = pm.sample_prior_predictive(1000)
    



In [ ]:
# Below model links home and away parameters to sigma and mean
# Not sure this is the correct way to do it

import pymc as pm

# Create Model
with pm.Model() as model:
    home = pm.Normal("home", 0, 0.5)

    # Attack ratings for teams - common mu and sd that all teams draw from
    atts_mu = pm.Normal("atts_mu", 0, 0.5)
    atts_sd = pm.HalfNormal("atts_sd", 0.5)
    atts = pm.Normal("atts", mu=atts_mu, sigma=atts_sd, shape=n_teams)

    # Defence ratings for teams - common mu and sd that all teams draw from
    defs_mu = pm.Normal("defs_mu", 0, 0.5)
    defs_sd = pm.HalfNormal("defs_sd", 0.5)
    defs = pm.Normal("defs", mu=defs_mu, sigma=defs_sd, shape=n_teams)

    # Calculate theta
    #home_theta = home + atts[home_team] + defs[away_team]
    #away_theta = atts[away_team] + defs[home_team]

    home_theta = pm.math.exp(home + atts[home_team] + defs[away_team])
    away_theta = pm.math.exp(atts[away_team] + defs[home_team])

    # Register away_theta and home_theta as a deterministic variable
    away_theta_det = pm.Deterministic("away_theta", away_theta)
    home_theta_det = pm.Deterministic("home_theta", away_theta)

    # Log-linear model for sigma_home_points - must be positive
    # Factors affecting sigma_home_points (variance): home + atts[home_team] + defs[away_team]
    # Log scale centered at 0 implies untransformed sigma home points starts at 1
    # then e-3 to e+3 is 0.05 to 20 
    log_sigma_home_points = pm.Normal("log_sigma_home_points", 0, 1)
    sigma_home_points = pm.Deterministic(
        "sigma_home_points",
        pm.math.exp(
            log_sigma_home_points
            + home
            + atts[home_team]
            + defs[away_team]
        ),
    )

    # Similarly, for sigma_away_points
    log_sigma_away_points = pm.Normal("log_sigma_away_points", 0, 1)
    sigma_away_points = pm.Deterministic(
        "sigma_away_points",
        pm.math.exp(
            log_sigma_away_points
            + atts[away_team]
            + defs[home_team]
        ),
    )

    # Truncated Normal for observed scores
    home_points = pm.Truncated(
        "home_points",
        pm.Normal.dist(mu=home_theta, sigma=sigma_home_points),
        lower=0,
        observed=home_score_arr,
    )
    away_points = pm.Truncated(
        "away_points",
        pm.Normal.dist(mu=away_theta, sigma=sigma_away_points),
        lower=0,
        observed=away_score_arr,
    )
    
    # Get prior samples
    prior_samples = pm.sample_prior_predictive(1000)


In [ ]:
# Plot chart
import matplotlib.pyplot as plt
import arviz as az

# The below range looks more appropriate - however there is still negative values

set_obs_number = 1

# Extract the value for the line from away_score_arr[0]
value = away_score_arr[set_obs_number]

# Plot simulated values for first observation
az.plot_dist(
    post_samples.posterior_predictive['away_points'][:,:,set_obs_number].values,
    kind="hist",
    hist_kwargs={"alpha": 0.6},
    label="simulated",
)

# Add a vertical line for away_score_arr[0]
plt.axvline(x=value, color='red', linestyle='--', label=f"Actual data")

# Add labels and legend
plt.xlabel("Away Points")
plt.ylabel("Frequency")
plt.legend()
plt.title(f"Actual data: {value}")
plt.show()


# Plot all observations
# Plot simulated values for first observation
az.plot_dist(
    post_samples.posterior_predictive['away_points'].values,
    kind="hist",
    hist_kwargs={"alpha": 0.6,"density": True},
    label="simulated",
)

az.plot_dist(
    away_score_arr,
    kind="hist",
    color="C1",
    hist_kwargs={"alpha": 0.6,"density": True},
    label="Actual",
)


# Add labels and legend
plt.title("All obs")
plt.xlabel("Away Points")
plt.ylabel("Frequency")
plt.legend()
plt.show()
